# Base CNN model from NVIDIA paper


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

PROJECT_ROOT = '/content/drive/MyDrive/cs-7643 project v2'
CODE_DIRECTORY = 'NVIDIA Dave Starter Code'

# Checking that files are accessible for the current user.
if not CODE_DIRECTORY in os.listdir(PROJECT_ROOT):
    # If this message displays, please create a shortcut to the group shared folder.
    print("Unable to access project files.")
else:
    print("Project files are accessible.")

In [ ]:
# Uncomment the command below if there is some unforeseen need to unpack the compressed dataset again.
# !unzip "/content/drive/MyDrive/cs-7643 project v2/NVIDIA Dave Starter Code/data/dataset.zip" -d "/content/drive/MyDrive/cs-7643 project v2/NVIDIA Dave Starter Code/data"

In [1]:
# Switching to the code directory to allow imports and access to dataset.
# %cd "/content/drive/MyDrive/cs-7643 project v2/NVIDIA Dave Starter Code"
!pwd

/home/jupyter/src/gcs


In [ ]:
# ! pip install ipywidgets
# ! jupyter nbextension enable --py widgetsnbextension

In [16]:
import copy
import time
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim
import torchvision.transforms as transforms

from utils import dataset
from models import nvidia

# Tqdm progress bar
from tqdm import tqdm_notebook, tqdm
import sigopt

LEARNING_RATE = 1e-4
MOMENTUM = 0.7
WEIGHT_DECAY_REGULARIZATION_TERM = 1e-5  # aiming for overfitting
BATCH_SIZE = 32
NUM_EPOCHS = 5


# Citation:
# - AverageMeter taken verbatim from the Assignment 2 training code.
# - Remainder of code in this file based on Assignment 2 training code.

class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def train(epoch, data_loader, model, optimizer, criterion, scaler=None):
    iter_time = AverageMeter()
    losses = AverageMeter()
    

    # Get the progress bar for later modification
    progress_bar = tqdm_notebook(data_loader, ascii=True)

    for idx, (data, target) in enumerate(progress_bar):
        start = time.time()

        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()

        # Forward pass and computation of loss.
        with torch.autocast("cuda"): #Automatic Mixed precision
            out = model(data).reshape(target.shape)
            # MSE loss
            loss = criterion(out, target)

        
        # Backwards pass to determine gradients and update model parameters.
        # optimizer.zero_grad()
        # loss.backward()
        # optimizer.step()

        for param in model.parameters():
            param.grad = None
        scaler.scale(loss).backward()
        scaler.step(optimizer)

        scaler.update()
        losses.update(loss, out.shape[0])

        iter_time.update(time.time() - start)
        # if idx % 5 == 0:
        #     print(('Epoch: [{0}][{1}/{2}]\t'
        #            'Time {iter_time.val:.3f} ({iter_time.avg:.3f})\t'
        #            'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
        #           .format(epoch, idx, len(data_loader), iter_time=iter_time, loss=losses)))

        progress_bar.set_description_str(f"Epoch {epoch}, Batch: {idx+1}, Loss: {loss.item():.4f}")
            # "Batch: %d, Loss: %.4f" % ((idx + 1), loss.item()))
        
    return losses.avg

def validate(epoch, validation_loader, model, criterion):
    iter_time = AverageMeter()
    losses = AverageMeter()

    # Get the progress bar for later modification
    progress_bar = tqdm_notebook(validation_loader, ascii=True)

    for idx, (data, target) in enumerate(progress_bar):
        start = time.time()

        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()

        out = None
        loss = None

        with torch.no_grad():
            out = model(data)
            # MSE loss
            loss = criterion(out, target)

        # loss.squeeze()
        losses.update(loss, out.shape[0])

        iter_time.update(time.time() - start)

#         if idx % 10 == 0:
#             print(('Epoch: [{0}][{1}/{2}]\t'
#                    'Time {iter_time.val:.3f} ({iter_time.avg:.3f})\t')
#                   .format(epoch, idx, len(validation_loader), iter_time=iter_time, loss=losses))

        progress_bar.set_description_str(f"Batch: {idx+1}, Loss: {loss.item():.4f}")
           
        
        # progress_bar.set_description_str(
        #     "Batch: %d, Loss: %.4f" % ((idx + 1), loss.item()))

    print("* Average Loss @1: {loss.avg:.4f}".format(loss=losses))
    return losses.avg

# RSME Loss function. eps prevents [nan] in the backward pass
def RSMELoss(yhat, y, eps=1e-6):
    return torch.sqrt(torch.mean((yhat-y)**2) + eps)

def plots(losses, lr=LEARNING_RATE, reg=WEIGHT_DECAY_REGULARIZATION_TERM, batch=BATCH_SIZE, momentum=MOMENTUM):
    fig1, ax1 = plt.subplots(figsize=(8, 10))
    ax1.plot(losses[0], label='Training Losses')
    ax1.plot(losses[1], label='Validation Losses')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Loss')
    ax1.set_title(f'CNN Loss Curve')
    ax1.legend(loc="best")
    plt.grid()
    plt.show()



In [17]:
# !! Skip this cell for sigopt experimentation !!
def main():
    # Normalizing images per the paper and resizing each image to 66 x 200.
    transform = transforms.Compose([
        # Citation:
        # https://pytorch.org/vision/stable/transforms.html#scriptable-transforms
        transforms.Resize((192, 256)),
        transforms.RandomRotation(degrees=5)
    ])

    # Loading in images with normalization and resizing applied.
    training_set, validation_set = dataset.load_nvidia_dataset(transform=transform, batch_size=BATCH_SIZE)
    torch.cuda.empty_cache()

    # Loading in the NVIDIA DAVE-2 model.
    model = nvidia.NvidiaDaveCNN()

    if torch.cuda.is_available():
        model = model.to(torch.device("cuda"))


    # Specify Mean Squared Error (MSE) or RSME as the criterion since this is a regression task.
    criterion = nn.MSELoss()
    # criterion = RMSELoss

    # Using Stochastic Gradient Descent (SGD) as the optimizer.
    optimizer = torch.optim.Adam(model.parameters(), 
                                LEARNING_RATE,
                                weight_decay=WEIGHT_DECAY_REGULARIZATION_TERM)
    
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=1, verbose=True) 
    # torch.backends.cudnn.benchmark = True

    best = float('inf')
    best_model = None
    best_epoch = None
    train_losses = []
    valid_losses = []
    scaler = torch.cuda.amp.GradScaler()
    for epoch in range(NUM_EPOCHS):

        # Training.
        train_loss = train(epoch, training_set, model, optimizer, criterion, scaler=scaler)
        temp = train_loss
        temp.cpu()
        temp = float(temp)
        train_losses.append(temp)  # average losses
        print(f"Average training loss this epoch: {temp:.4f}")

        # Validation.
        valid_loss = validate(epoch, validation_set, model, criterion)
        temp = valid_loss
        temp.cpu()
        temp = float(temp)
        valid_losses.append(temp)

        scheduler.step(train_loss)

        if valid_loss < best:
            best = valid_loss
            best_model = copy.deepcopy(model)
            best_epoch = epoch
        print(f"Best performing model so far average validation loss: {best:.4f} on epoch {best_epoch}\n")

    print('Best Training Loss @1: {:.4f}'.format(best))

    torch.save(best_model.state_dict(), './checkpoints/nvidia_dave2.pth')

    losses_to_plot = train_losses, valid_losses
    plots(losses_to_plot, lr=LEARNING_RATE, reg=WEIGHT_DECAY_REGULARIZATION_TERM,
          batch=BATCH_SIZE, momentum=MOMENTUM)

    

if __name__ == '__main__':
    main()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:52: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/846 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [18]:
def evaluate(args):       
        
    # log source of hyperparameter suggestion
    sigopt.log_metadata('optimizer', "Adam")
    sigopt.log_model("CNN - NVIDIA inspired")
    sigopt.log_dataset("Udacity self-driving dataset ")
    
    sigopt.params.setdefault("learning_rate", args['learning_rate'])
    sigopt.params.setdefault("reg", args['reg'])
    sigopt.params.setdefault("batch_size", int(args['batch_size']))
    
    # Normalizing images per the paper and resizing each image to 66 x 200.
    transform = transforms.Compose([
        # Citation:
        # https://pytorch.org/vision/stable/transforms.html#scriptable-transforms
        transforms.Resize((192, 256)),
        transforms.RandomRotation(degrees=5)
        
    ])

    print("Params : ", sigopt.params)
    # Loading in images with normalization and resizing applied.
    training_set, validation_set = dataset.load_nvidia_dataset(transform=transform, batch_size=int(sigopt.params.batch_size))
    torch.cuda.empty_cache()

    # Loading in the NVIDIA DAVE-2 model.
    model = nvidia.NvidiaDaveCNN()

    if torch.cuda.is_available():
        model = model.to(torch.device("cuda"))


    # Specify Mean Squared Error (MSE) as criterion since this is a regression task. (We ultimately take sqrt and convert it to RMSE)
    criterion = nn.MSELoss()

    # Using Stochastic Gradient Descent (SGD) as the optimizer.
    optimizer = torch.optim.Adam(model.parameters(), sigopt.params.learning_rate,
                                weight_decay=sigopt.params.reg)
    
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=1, verbose=True) 
    # torch.backends.cudnn.benchmark = True

    best = float('inf')
    best_model = None
    best_epoch = None
    train_losses = []
    valid_losses = []
    scaler = torch.cuda.amp.GradScaler()
    for epoch in range(NUM_EPOCHS):

        # Training.
        train_loss = train(epoch, training_set, model, optimizer, criterion, scaler=scaler)
        temp = train_loss
        temp.cpu()
        temp = float(temp)
        train_losses.append(temp)  # average losses
        print(f"Average training loss this epoch: {temp:.4f}")

        # Validation.
        valid_loss = validate(epoch, validation_set, model, criterion)  # JTS - changed from test to validation
        temp = valid_loss
        temp.cpu()
        temp = float(temp)
        valid_losses.append(temp)

        # scheduler.step(train_loss)  # JTS - this may or may not interfere with sigopt

        if valid_loss < best:
            best = valid_loss
            best_model = copy.deepcopy(model)
            best_epoch = epoch
        print(f"Best performing model so far average validation loss: {best:.4f} on epoch {best_epoch}\n")

    print('Best Training Loss @1: {:.4f}'.format(best))

    torch.save(best_model.state_dict(), './checkpoints/nvidia_dave2.pth')
    sigopt.log_metric(name='MSE', value=best)
    return best.cpu().numpy()

In [19]:
import os
os.environ["SIGOPT_API_TOKEN"] = "XWBIVDWCVQXALUZQFDHNGOELLLKDJBMOJALEPCNQXQGBNIGC"
os.environ['SIGOPT_PROJECT'] = 'nvidia_cnn'
%reload_ext sigopt
args = {
    'batch_size': 256,
    'learning_rate': 0.005,
    'momentum': 0.9,
    'reg': 0.0005
}

In [20]:
%%experiment
{
    'name': 'GCP NVIDIA CNN model Optimization 3',
    'metrics': [
        {
            'name': 'MSE',
            'strategy': 'optimize',
            'objective': 'minimize',
        }
    ],
    'parameters': [
        {
            'name': 'reg',
            'type': 'double',
            'bounds': {'min': 0.0001, 'max': 0.5},
            'transformation': 'log'
        },
        {
            'name': 'learning_rate',
            'type': 'double',
            'bounds': {'min': 0.0001, 'max': 0.9},
            'transformation': 'log'
        },
        {
            'name': 'batch_size',
            'type': 'categorical',
            'categorical_values': ['32', '64']
        }
    ],
    'budget': 20
}

Experiment created, view it on the SigOpt dashboard at https://app.sigopt.com/experiment/530307


In [21]:
%%optimize gcp_nvidia_cnn_optimization_run_3
evaluate(args)

Run started, view it on the SigOpt dashboard at https://app.sigopt.com/run/441508
Params :  {'reg': 0.0049732843553982175, 'learning_rate': 0.0013222610305548663, 'batch_size': '32'}


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:52: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/846 [00:00<?, ?it/s]

Average training loss this epoch: 0.0640


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:98: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

* Average Loss @1: 0.0759
Best performing model so far average validation loss: 0.0759 on epoch 3



  0%|          | 0/423 [00:00<?, ?it/s]

Average training loss this epoch: 0.0732


  0%|          | 0/106 [00:00<?, ?it/s]

* Average Loss @1: 0.0758
Best performing model so far average validation loss: 0.0758 on epoch 4

Best Training Loss @1: 0.0758
Run finished, view it on the SigOpt dashboard at https://app.sigopt.com/run/441851
